In [ ]:
# Skriptas sukuria zive duomenų analogą iš MIT duomenų, į sub-aplanką rec_dir įrašomi
# transformuoti EKG įrašai *.npy ir anotacijų failai *.json.
# Išeities duomenys MIT_BIH atsisiunčiami iš aplanko mit-bih-arrhythmia-database-1.0.0
# https://archive.physionet.org/physiobank/database/html/mitdbdir/mitdbdir.htm
# The recordings were digitized at 360 samples per second per channel with 11-bit
# resolution over a 10 mV range. In most records, the upper signal (channel 0) is
# a modified limb lead II (MLII), obtained by placing the electrodes on the chest.

# Iš MIT reaguojame tik į anotacijas N,L,R,e,j,V,E,S,A,a,J,F,Q, visas kitas ignoruojame.
# MIT2ZIVE duomenyse paliekame tik makroanotacijas: N,S,V,F,U. 

# Failai skaldomi į 3 dalis, recordId = 1,2,3, kad būtų panašaus ilgio, kauip Zive įrašai.

# Programos planas:
# Nuskaitome MIT-BIH ekg įrašą: channel = 0, skaitome mlvoltus
# Anotacijų mapingas į N,V,S,F,U
# Pakeičiame diskretizavimo dažnį iš 360 į 200 Hz
# Perskaičiuojame anotacijos vietas
# Padalijame įrašus į tris dalis
# Įrašome ekg ir anotacijas zive formatu į diską
# Suformuojame comments.csv failą, kuriame komentaruose nurodysime DS1, ar DS2 

# //////////////////////////////////////////////////////////////////////////////////////////////////////
#  
import pandas as pd
import numpy as np
import wfdb
# Waveform Database Software Package (WFDB) for Python
# https://physionet.org/content/wfdb-python/4.1.0/

import json
import sys
from pathlib import Path
from neurokit2 import signal_resample
from collections import Counter
import datetime

# from zive_util_vu import create_dir
from mit_bih_util import create_dir

def make_dict_array(atr_sample, atr_symbol):
    array_of_dict = []
    for sample, symbol in zip(atr_sample, atr_symbol):
        d = {'sample': sample, 'symbol': symbol}
        array_of_dict.append(d)
    return array_of_dict   

def print_dict_array(dict_array, incl = []):
    for d in dict_array:
        sample = d['sample']
        symbol = d['symbol']
        if bool(incl) == False:
            if (symbol != 'N'):
                print(f"Sample: {sample}, Symbol: {symbol}")    
        else:
            if (symbol in incl):
                print(f"Sample: {sample}, Symbol: {symbol}")     

def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.10*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

def zive_read_df_rpeaks(db_path, file_name):
    file_path = Path(db_path, file_name + '.json')
    with open(file_path,'r', encoding="utf8") as f:
        data = json.loads(f.read())
    df_rpeaks = pd.json_normalize(data, record_path =['rpeaks'])
    return df_rpeaks

def get_noise_spans(atr_sample, atr_symbol):
    occurrences = [i for i in range(len(atr_symbol)) if atr_symbol[i] == '~']
    # print(occurrences)
    pairs = []
    for i in range(0, len(occurrences), 2):
        if i+1 < len(occurrences):
            startIndex = atr_sample[occurrences[i]]
            endIndex = atr_sample[occurrences[i+1]]
            pairs.append({'startIndex': int(startIndex), 'endIndex': int(endIndex)})
    return pairs


print("\nSukuriamas Zive duomenų analogas iš MIT duomenų")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI\\DUOM_2022_RUDUO'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/DUOMENU_TVARKYMAS_2023'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Aplankas su MIT-BIH duomenų rinkiniu
db_folder_mit = 'mit-bih-arrhythmia-database-1.0.0'

#  Aplankas, kur rašome MIT2ZIVE duomenis
db_folder_mit2zive = 'records_npy_tst'


# Užduodamas pacientų įrašų sąrašas
# Testavimui
records_nr = np.array([116])
# records_nr = np.array([124,209,228])

records_nr = np.array(
[100, 101, 103, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
200, 201, 202, 203, 205, 208, 209, 210, 212, 213, 214, 215, 219, 220, 221, 222, 223, 228, 230, 231,
232, 233, 234])

records_nr = np.array([116])

# MIT_BIH duomenų diskretizavimo dažnumas
fs_mit = 360



# ////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT-BIH duomenų rinkinį
db_path_mit = Path(Duomenu_aplankas, db_folder_mit)

print("Bendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("MIT duomenų aplankas: ", db_folder_mit)
print("Pacientų įrašų sąrašas:\n",records_nr)
print("Aplankas, kur rašome MIT2ZIVE duomenis:\n", db_folder_mit2zive)

print("Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers ir yra praleisti")
print("207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF")
print("\nDiskretizavimo dažnis fs_mit: ", fs_mit)


# //////////////// Nurodomi MIT2ZIVE parametrai ////////////////////////////////


# MIT2ZIVE duomenų diskretizavimo dažnumas
fs_zive = 200 # diskretizavimo dažnumas

# Neignoruojamų anotacijų sąrašas
annot_list = ['N','L','R','e','j','A','a','J','S','V','E','F','Q','~']

# Failai pūpsnių makro anotacijų formavimui
annot_grouping = {
'N':'N','R':'N', 'L':'N', 'e':'N', 'j':'N', 'A':'S','a':'S', 'J':'S', 'S':'S', 'V':'V', 'E':'V', 'F':'F','Q':'U', '~':'~' }


selected_beats = {'N':0, 'S':1, 'V':2, 'F':3, 'U':4}

# ////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT2ZIVE duomenų rinkinį
# db_path_mit2zive = Path(Duomenu_aplankas, db_folder_mit2zive)

# Nuoroda į aplanką MIT2ZIVE EKG įrašams (.npy) ir anotacijoms (.json)
rec_dir = Path(Duomenu_aplankas, db_folder_mit2zive)

sant = fs_zive/fs_mit

print("\nAplankas transformuotiems duomenims: ", db_folder_mit2zive)

# Sukūriame aplanką transformuotiems iš MIT į Zive formatą duomenims
create_dir(rec_dir)

print("Diskretizavimo dažnis fs_zive: ", fs_zive)
print(f"fs_zive/fs_mit = {sant:.2f}")

print("Paliekamos įrašuose anotacijos:", annot_list)
print("Failas grupavimui į makroanotacijas:", annot_grouping)

        

In [ ]:

# Ciklas per pacientų įrašus
for record_nr in records_nr:

# ----------------------------------------------- Nuskaitome įrašą
    subject_path = f'{db_path_mit}/{record_nr}'
    # ic(subject_path)

    # Įrašo skaitymo variantai
    # sequence, fields = wfdb.rdsamp(subject_path, channels=[0])
    # record = wfdb.rdrecord('sample-data/a103l')

    # https://www.programmersought.com/article/28613723297/
    
     # Variantas 1 - skaitmeninės reikšmės
    # record = wfdb.rdrecord(subject_path, sampfrom=1,
    #                             channels=[0], physical=False)
    # sign_raw = record.d_signal[:,0]

    # Variantas 2 - fizinės reikšmės - perverstos į mV
    
    record = wfdb.rdrecord(subject_path, sampfrom=0,channels=[0], physical=True)
    signl_raw = record.p_signal[:,0]
    # print(sign_raw[:20])

    # https://wfdb.readthedocs.io/en/latest/wfdb.html

    len_signl_raw = signl_raw.shape[0]
    print("\n")
    print(f"Pacientas iš MIT: {record_nr}  Reikšmių: {len_signl_raw}")

    # Eilutės galimam filtravimui
    # sign_transf = sign_raw
    # sign_transf = signal_filter(signal=sign_raw, sampling_rate=360,
    #  lowcut=0.1, method="butterworth", order=5)

    # Nuskaitome originalaus įrašo anotacijas
    # https://wfdb.readthedocs.io/en/latest/wfdb.html 
    ann = wfdb.rdann(subject_path, 'atr', sampfrom=0, sampto=None, shift_samps=False)
    atr_sample_org = ann.sample
    atr_symbol_org = np.array(ann.symbol)

    print("\nAnotacijų pasiskirstymas originale:")
    smb_lst = Counter(atr_symbol_org)
    print(dict(smb_lst))
    print()
    # smb_lst = Counter(atr_symbol)
    # print("\nOriginalių anotacijų pasiskirstymas:")
    # print(smb_lst)
    # print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)


# ------------------------------------ Resampling iš 360 Hz (MIT) į 200 Hz (Zive)
    signl = signal_resample(signl_raw, sampling_rate=fs_mit,
                              desired_sampling_rate=200, method="numpy")
    # https://neurokit2.readthedocs.io/en/latest/_modules/neurokit2/signal/signal_resample.html

    signl_len = signl.shape[0]
    print(f"Pacientas Zive rinkiniui: {record_nr}  Resamplintas. Reikšmių: {signl_len}")

    # print(signl[:20])

    # Resampliname anotacijų vietas
    atr_sample_org = (atr_sample_org*sant).astype(int)

    print("\nAnotacijų pasiskirstymas po resampling:")
    smb_lst = Counter(atr_symbol_org)
    print(dict(smb_lst))
    # print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array, incl = ['~'])

# ------------------------------------------------- Sukūriame makro anotacijas
    # Paliekame tik užduotas anotacijas iš annot_list, visas kitas (tarnybines)
    # išvalome, išskyrus `~`, jas išvalysime vėliau 
    # Tarnybinės yra: '+', ir visos kitos
    
    atr_sample_n = []
    atr_symbol_n = []
    for i in range(atr_sample_org.shape[0]):
        if atr_symbol_org[i] not in annot_list:
            continue
        else:
            atr_sample_n.append(atr_sample_org[i])
            atr_symbol_n.append(atr_symbol_org[i])
    atr_sample = np.array(atr_sample_n)
    atr_symbol = np.array(atr_symbol_n)

    print("\nAnotacijų pasiskirstymas po anotacijų valymo:")
    smb_lst = Counter(atr_symbol)
    print(dict(smb_lst))
    print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)

    # Anotacijas stambinam į makro anotacijas
    for i in range(len(atr_symbol)):
        if atr_symbol[i] in annot_grouping:
            atr_symbol[i] = annot_grouping[atr_symbol[i]]

    print("Makro anotacijų pasiskirstymas:")
    smb_lst = Counter(atr_symbol)
    print(dict(smb_lst))
    # print()
    # dict_array = make_dict_array(atr_sample, atr_symbol)
    # print_dict_array(dict_array)

# -------------------------------------- Sudaliname įrašus ir anotacijas į n dalių ir įrašome į diską
    
    n = 3 # dalių, į kurias daliname įrašus, skaičius
    # signl_len = 10 # testavimui
    smb_lst_suminis = {'N': 0, 'S': 0, 'V': 0, 'F':0, 'U':0, '~':0 } # testavimui
    
    # Pasiruošimas
    len_sub_record = signl_len//n # will be assigned the value of the integer quotient of dividing len_sign by n
    # print(n, signl_len, len_sub_record, len_sub_record*n)

    # Pasiruošiame įrašų dalinimui triukšmų lentelę
    # noise_table = convert_noise_table(noise_table_init, len_sub_record)
    # if (record_nr == 100):
    #     for elem in noise_table:
    #         print(elem)

    print("\nĮrašo ir anotacijų dalijimas į n dalių")
    # Ciklas per n sub_records
    for sub_recId in range(n):
        print('\nsub_recId:', sub_recId, "Įrašo ilgis:", len_sub_record)
        lowerIndex = len_sub_record*sub_recId
        upperIndex = len_sub_record*(sub_recId+1) - 1
        # print('lowerIndex:', lowerIndex, 'upperIndex:', upperIndex, 'upperIndex-lowerIndex+1:', upperIndex-lowerIndex+1)

        # deviding signal array
        sub_record = np.empty(len_sub_record)
        sub_record = signl[lowerIndex:upperIndex+1]
        # print("len of sub_record:", len(sub_record))

        # Suformuojame padalinto EKG įrašo failo vardą (pvz. iš '100' padarome 10001.001) 
        rec_sub = '{:02d}'.format(sub_recId+1)
        rec_ext = '{:03d}'.format(sub_recId+1)
        file_name = str(record_nr) + rec_sub + '.' + rec_ext

        # Įrašome suformuotą sub_record į duomenų aplanką
        file_path = Path(rec_dir, file_name)
        with open(file_path, 'wb') as f:
            np.save(f, sub_record)
        # print('file_path:', file_path)
        
        # deviding annotation array
        sub_atr_sample = []
        sub_atr_symbol = []
        for i in range(len(atr_sample)):
            if (atr_sample[i] >= lowerIndex) & (atr_sample[i] < upperIndex):
                sub_atr_sample.append(atr_sample[i] - lowerIndex)
                sub_atr_symbol.append(atr_symbol[i])

        counter_obj = Counter(sub_atr_symbol) 
        annot_items = counter_obj.items()

        # testavimui
        print("Makro anotacijų pasiskirstymas:")
        dict_obj = dict(counter_obj)
        print(dict_obj)
        # print()
        for key in dict_obj:
            # smb = elem[0]
            val = dict_obj[key]
            smb_lst_suminis[key] = smb_lst_suminis[key] + val
        print('smb_lst_suminis:', smb_lst_suminis)    
        # print()

        # Analizei taip pat sudalijame į dalis taip pat ir originalius atributus,
        # kurie nėra išvalyti nuo tarnybinių anotacijų

        # sub_atr_sample_org = []
        # sub_atr_symbol_org = []
        # for i in range(len(atr_sample_org)):
        #     if (atr_sample_org[i] >= lowerIndex) & (atr_sample_org[i] < upperIndex):
        #         sub_atr_sample_org.append(atr_sample_org[i] - lowerIndex)
        #         sub_atr_symbol_org.append(atr_symbol_org[i])

        # counter_obj = Counter(sub_atr_symbol_org) 
        # annot_items = counter_obj.items()

        # testavimui
        # print("\nVisų anotacijų pasiskirstymas:")
        # dict_obj = dict(counter_obj)
        # print(dict_obj)
        # dict_array = make_dict_array(sub_atr_sample_org,sub_atr_symbol_org)
        # print_dict_array(dict_array, incl = ['~'])
        # print()

        # for key in dict_obj:
        #     val = dict_obj[key]
        #     smb_lst_suminis[key] = smb_lst_suminis[key] + val
        # print('smb_lst_suminis:', smb_lst_suminis)    
        # print()


        # Paruošiame informaciją json failui

        recordingId = str(record_nr) + rec_sub

        rpeaks = []
        for idx in range(len(sub_atr_sample)):
            elem = {"sampleIndex": int(sub_atr_sample[idx]),
                    "annotationValue": sub_atr_symbol[idx]}
            rpeaks.append(elem)    
        print('rpeaks length:', len(rpeaks))

        # noises = get_noise_intervals(noise_table, recordingId)
        noises = get_noise_spans(sub_atr_sample, sub_atr_symbol)
        print('noises:', noises)

        json_data = {
            'recordingId': recordingId,
            "userId": str(record_nr),
            "rpeakAnnotationCounts": dict(annot_items),
            "noises": noises,
            "rpeaks": rpeaks 
        }

        # Įrašome json failą su įrašo atributais
        file_path = Path(rec_dir, file_name + '.json')
        with open(file_path, 'w') as f:
            json.dump(json_data, f)

    print('smb_lst_suminis:', smb_lst_suminis)    



In [ ]:
# ----------------------------------------------------------------------  SKAITYMO TESTAS 

print("\nKontrolinis skaitymas") 

# Kontrolinis  skaitymas

for sub_recId in range(n):
    rec_sub = '{:02d}'.format(sub_recId+1)
    rec_ext = '{:03d}'.format(sub_recId+1)
    file_name = str(record_nr) + rec_sub + '.' + rec_ext
   
    # Įrašas
    file_path = Path(rec_dir, file_name)
    # print('\nfile_path:', file_path)
    with open(file_path, "rb") as f:
        sign_loaded = np.load(f) 
        # print(sign_loaded[:10])
        # print('sign_loaded', sign_loaded.shape)
   
    # json
    # Nuskaitome gydytojo koreguotus EKG įrašo atributus iš įrašo json 
    df_rpeaks = zive_read_df_rpeaks(rec_dir, file_name)
    rpeaks_from_json = df_rpeaks['sampleIndex'].to_numpy()
    symbols_from_json = df_rpeaks['annotationValue'].to_numpy()
    # print(rpeaks_from_json[:20], symbols_from_json[:20])

    print("Makro anotacijų pasiskirstymas. Variantas iš json failo:")
    smb_lst = Counter(symbols_from_json)
    # print(dict(smb_lst))
    # print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)


print("\nPabaiga.............")


In [ ]:
# Paruošiame šabloną comments.csv failui
# pvz.
# filename,userId,recordingId,nesutmp,quality,comment
# 1626934.963,0,0,"Kokybė puiki, išskyrus pabaigą"

dict_array = []
for record_nr in records_nr:
    # print(record_nr)
    for sub_recId in range(n):
        rec_sub = '{:02d}'.format(sub_recId+1)
        recordingId = str(record_nr) + rec_sub
        rec_ext = '{:03d}'.format(sub_recId+1)
        file_name = recordingId + '.' + rec_ext
        elem = {'filename':file_name,  'userId': str(record_nr), 'recId': recordingId, 'nesutmp': 0, 'quality': 0, 'comment': ''}
        dict_array.append(elem)

df = pd.DataFrame(dict_array)
print(df.head(5))
file_name = 'comments_sablonas.csv'
file_path = Path(rec_dir, file_name)
df.to_csv(file_path, index=False)
print("Šablonas įrašytas į: ", file_path)




In [ ]:
# Dar vienas bandymas surinkti triukšmų intervalus, šį kartą pagal `~`

# Testavimui
records_nr = np.array([116])

for record_nr in records_nr:

# ----------------------------------------------- Nuskaitome įrašą
    subject_path = f'{db_path_mit}/{record_nr}'
    # ic(subject_path)

    record = wfdb.rdrecord(subject_path, sampfrom=0,channels=[0], physical=True)
    signl_raw = record.p_signal[:,0]
    # print(sign_raw[:20])

    len_signl_raw = signl_raw.shape[0]
    print("\n")
    print(f"Pacientas iš MIT: {record_nr}  Reikšmių: {len_signl_raw}")

    ann = wfdb.rdann(subject_path, 'atr', sampfrom=0, sampto=None, shift_samps=False)
    
    print("\nAnnotation object:")
    print(type(ann.sample))
    print(type(ann.symbol))
    pairs = get_noise_spans(list(ann.sample), ann.symbol)
    print(pairs)
    # for pair in pairs:
    #     pair[]

In [ ]:
# Triukšmų vietų perskaičiavimas - testas
# Naudojamas atvejui, kai naudotos pažymėtos vietos iš: Įrašų aprašymai
# From <https://archive.physionet.org/physiobank/database/html/mitdbdir/records.htm> 
# ir suvęstos rankiniu būdu į 'noise table'
# Dabar šis variantas nenaudojamas, vietoj jo naudojamas automatinis su žymėmis `~`

def convert_noise_table(noise_table_init, len_sub_record): 
# perdaro pradinę informaciją apie triukšmus gautą originaliems
# MIT įrašams į informaciją, pritaikytą resamplintams įrašams ir
# padalintiems į kelias dalis

    noise_table = []

    for elem in noise_table_init:
        # print(elem)
        noise_start = datetime.timedelta(minutes=elem['noise_min'], seconds=elem['noise_sec'])
        start_seconds = noise_start.total_seconds()
        start_samples = ((360*start_seconds)*sant)

        # print(start_samples, len_sub_record)
        q, r = divmod(start_samples, len_sub_record)
        startIndex = int(r)
        duration = int(elem['duration']*200)
        endIndex = startIndex + duration
        sub_recId = int(q)
        # print(f"rec: {elem['rec']}  sub_recId: {sub_recId+1} startIndex: {startIndex} endIndex: {endIndex}")

        rec_sub = '{:02d}'.format(sub_recId+1)
        new_rec = {'rec': str(elem['rec']) + rec_sub, 'startIndex': startIndex, 'endIndex': endIndex }
        noise_table.append(new_rec)
    return noise_table

def get_noise_intervals(noise_table, recordingId):
    # print(recordingId)
    filtered_list = [item for item in noise_table if item['rec'] == recordingId]
    noise_intervals = []
    for elem in filtered_list:
        row = {
            "startIndex": int(elem['startIndex']),
            "endIndex":  int(elem['endIndex'])
        }
        noise_intervals.append(row)
    return noise_intervals

# ----------------------------------------------------------------------------
noise_table_init = [
    # rec 100 - švarus
    {'rec':101, 'noise_min': 1, 'noise_sec': 48, 'duration': 10 },
    {'rec':101, 'noise_min': 5, 'noise_sec': 13, 'duration': 10 },
    
    # rec 102 - švarus
    # rec 103 - švarus

    {'rec':104, 'noise_min': 5, 'noise_sec': 13, 'duration': 10 },
    {'rec':104, 'noise_min': 6, 'noise_sec': 17, 'duration': 10 },
    
    {'rec':105, 'noise_min': 22, 'noise_sec': 2, 'duration': 10 },
    {'rec':105, 'noise_min': 27, 'noise_sec': 27, 'duration': 10 },
    {'rec':105, 'noise_min': 28, 'noise_sec': 8, 'duration': 10 },
    {'rec':105, 'noise_min': 29, 'noise_sec': 7, 'duration': 10 },

    # rec 106 - švarus
    {'rec':107, 'noise_min': 20, 'noise_sec': 38, 'duration': 10 },
    
    {'rec':108, 'noise_min': 28, 'noise_sec': 10, 'duration': 10 }, # triukšmingas
    {'rec':108, 'noise_min': 29, 'noise_sec': 0, 'duration': 10 },
    
    {'rec':109, 'noise_min': 5, 'noise_sec': 27, 'duration': 10 },
    {'rec':109, 'noise_min': 26, 'noise_sec': 9, 'duration': 10 },
    
    {'rec':111, 'noise_min': 3, 'noise_sec': 52, 'duration': 10 },
    {'rec':111, 'noise_min': 15, 'noise_sec': 41, 'duration': 10 },
    {'rec':111, 'noise_min': 19, 'noise_sec': 54, 'duration': 10 },
    {'rec':111, 'noise_min': 27, 'noise_sec': 2, 'duration': 10 },
    
    # rec 112 - švarus
    # rec 113 - švarus
    {'rec':114, 'noise_min': 20, 'noise_sec': 2, 'duration': 10 },
    # rec 115 - švarus
    {'rec':116, 'noise_min': 16, 'noise_sec': 37, 'duration': 10 },
    {'rec':116, 'noise_min': 23, 'noise_sec': 8, 'duration': 10 },

    {'rec':117, 'noise_min': 11, 'noise_sec': 58, 'duration': 10 },

    {'rec':118, 'noise_min': 8, 'noise_sec': 31, 'duration': 10 },
    {'rec':118, 'noise_min': 26, 'noise_sec': 48, 'duration': 10 },

    {'rec':119, 'noise_min': 20, 'noise_sec': 5, 'duration': 10 },
    {'rec':119, 'noise_min': 25, 'noise_sec': 33, 'duration': 10 },

    {'rec':121, 'noise_min': 24, 'noise_sec': 32, 'duration': 10 },
    {'rec':121, 'noise_min': 26, 'noise_sec': 1, 'duration': 10 },
    
    # rec 122 - švarus
    # rec 123 - švarus
    # rec 124 - švarus

    {'rec':200, 'noise_min': 5, 'noise_sec': 38, 'duration': 10 },
    {'rec':200, 'noise_min': 20, 'noise_sec': 52, 'duration': 11 },
    {'rec':201, 'noise_min': 18, 'noise_sec': 14, 'duration': 10 },
    # rec 202 - švarus

    {'rec':203, 'noise_min': 15, 'noise_sec': 2, 'duration': 10 },
    {'rec':203, 'noise_min': 23, 'noise_sec': 25, 'duration': 10 },
    {'rec':203, 'noise_min': 24, 'noise_sec': 46, 'duration': 10 },
    # rec 205 - švarus

    {'rec':207, 'noise_min': 5, 'noise_sec': 19, 'duration': 10 },
    {'rec':207, 'noise_min': 6, 'noise_sec': 56, 'duration': 10 },

    {'rec':208, 'noise_min': 14, 'noise_sec': 57, 'duration': 10 },
    {'rec':208, 'noise_min': 21, 'noise_sec': 6, 'duration': 10 },
    {'rec':208, 'noise_min': 23, 'noise_sec': 0, 'duration': 10 },

    {'rec':209, 'noise_min': 5, 'noise_sec': 50, 'duration': 10 },
    {'rec':209, 'noise_min': 28, 'noise_sec': 17, 'duration': 10 },

    {'rec':210, 'noise_min': 3, 'noise_sec': 47, 'duration': 10 },
    {'rec':210, 'noise_min': 15, 'noise_sec': 55, 'duration': 10 },

    {'rec':212, 'noise_min': 18, 'noise_sec': 42, 'duration': 10 },
    {'rec':212, 'noise_min': 26, 'noise_sec': 45, 'duration': 10 },

    # rec 213 - švarus
    # rec 214 - švarus

    {'rec':215, 'noise_min': 3, 'noise_sec': 11, 'duration': 10 },

    {'rec':217, 'noise_min': 14, 'noise_sec': 1, 'duration': 10 },
    # rec 219 - švarus, Atrial fibrillation
    # rec 220 - švarus
    
    {'rec':221, 'noise_min': 19, 'noise_sec': 42, 'duration': 10 },
   
    {'rec':222, 'noise_min': 24, 'noise_sec': 43, 'duration': 10 },

    # rec 223 - švarus

    {'rec':228, 'noise_min': 4, 'noise_sec': 35, 'duration': 10 },
    {'rec':228, 'noise_min': 20, 'noise_sec': 8, 'duration': 10 },
    # rec 230 - švarus
    # rec 231 - švarus

    {'rec':232, 'noise_min': 0, 'noise_sec': 58, 'duration': 10 },
    {'rec':232, 'noise_min': 11, 'noise_sec': 51, 'duration': 10 },
    # rec 233 - švarus

    {'rec':234, 'noise_min': 6, 'noise_sec': 37, 'duration': 10 },
    {'rec':234, 'noise_min': 23, 'noise_sec': 17, 'duration': 10 },
]

print("\nTestavimas naudojant noise table")

noise_table = convert_noise_table(noise_table_init, len_sub_record)
for elem in noise_table:
    print(elem)

for record_nr in records_nr:
    # print(record_nr)
    for sub_recId in range(n):
        rec_sub = '{:02d}'.format(sub_recId+1)
        recordingId = str(record_nr) + rec_sub
        noise_intervals = get_noise_intervals(noise_table, recordingId)
        if bool(noise_intervals) != False:
            print(noise_intervals)

In [ ]:
# Testas kokybės anotacijoms išgauti - nepasisekęs, nes kokybės anotacijos sq_ann = ann.aux_note tuščios,
# paaiškėjo, kad lightwave naudoja `~`

def get_orig_sample_from_time(minutes, seconds):
    noise_start = datetime.timedelta(minutes=minutes, seconds=seconds)
    start_seconds = noise_start.total_seconds()
    start_samples = int(360*start_seconds)
    return start_samples

def get_noise_place(atr_sample_org, atr_symbol_org, sample):
    for idx in range(len(atr_sample_org)):
        if atr_sample_org[idx] >= sample:
            place360 = atr_sample_org[idx] 
            place200 = (atr_sample_org[idx]*sant).astype(int)
            symbol = atr_symbol_org[idx]
            break
    return place360, place200, symbol


# Testavimui
records_nr = np.array([116])

for record_nr in records_nr:

# ----------------------------------------------- Nuskaitome įrašą
    subject_path = f'{db_path_mit}/{record_nr}'
    # ic(subject_path)

    record = wfdb.rdrecord(subject_path, sampfrom=0,channels=[0], physical=True)
    signl_raw = record.p_signal[:,0]
    # print(sign_raw[:20])

    len_signl_raw = signl_raw.shape[0]
    print("\n")
    print(f"Pacientas iš MIT: {record_nr}  Reikšmių: {len_signl_raw}")

    

    # Nuskaitome originalaus įrašo anotacijas
    # https://wfdb.readthedocs.io/en/latest/wfdb.html 

    # wfdb.io.rdann(record_name, extension, sampfrom=0, sampto=None, shift_samps=False, 
    # pn_dir=None, return_label_elements=['symbol'], summarize_labels=False)
    # Read a WFDB annotation file record_name.extension and return an Annotation obj

    ann = wfdb.rdann(subject_path, 'atr', sampfrom=0, sampto=None, shift_samps=False, 
            pn_dir=None, return_label_elements=['symbol'], summarize_labels=True)
    
    atr_sample_org = ann.sample
    atr_symbol_org = np.array(ann.symbol)
    
    print("\nAnotacijų pasiskirstymas originale:")
    smb_lst = Counter(atr_symbol_org)
    print(dict(smb_lst))
    print()


    print("\nKokybės anotacijos originale:")
    sq_ann = ann.aux_note
    print(sq_ann[:20])
    print('len(sq_ann):', len(sq_ann))


    print("\nKokybės anotacijų pasiskirstymas originale:")
    sq_ann_org = np.array(sq_ann)
    smb_lst = Counter(sq_ann_org)
    print(dict(smb_lst))
    print()


    smb_lst = Counter(atr_symbol_org)
    print("\nOriginalių anotacijų pasiskirstymas:")
    print(smb_lst)
    print()
    dict_array = make_dict_array(atr_sample_org,atr_symbol_org)
    print_dict_array(dict_array)
    

note = '16:37 Noise, saturation in upper signal'
sample = get_orig_sample_from_time(16, 37)
print('\n', note, 'start:', sample)
place360, place200, symbol = get_noise_place(atr_sample_org, atr_symbol_org, sample)
print('place360:', place360, 'place200:', place200, 'symbol:', symbol )


note = '23:08 Noise, saturation in upper signal'
sample = get_orig_sample_from_time(23, 8)
print('\n', note, 'start:', sample)
place360, place200, symbol = get_noise_place(atr_sample_org, atr_symbol_org, sample)
print('place360:', place360, 'place200:', place200, 'symbol:', symbol )
